In [29]:
import pandas as pd
import os

In [30]:
def create_df():
    csv_files = [f for f in os.listdir('inputs') if f.endswith(".csv")]
    csv_file = str(csv_files[0])
    df = pd.read_csv(f'inputs/{csv_file}')
    payments = df.shape[0]+1
    print(f'{payments} payments were found in this file')
    return df
df = create_df()

142 payments were found in this file


In [31]:
def discover_amounts(df):
    sections = ['Squirrels', 'Beavers', 'Cubs', 'Scouts']
    payment_schedules = ['Activities', 'Monthly Subscriptions']

    result_subsets = []  # List to store subsets

    df = df.groupby(['resources.description'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()

    for section in sections:
        subset_section = df[df['resources.description'].str.contains(section, case=False)].copy()

        for schedule in payment_schedules:
            subset_schedule = subset_section[subset_section['resources.description'].str.contains(schedule, case=False)].copy()
            subset_schedule['Section Payment Schedule'] = section + ": " + schedule

            result_subsets.append(subset_schedule)

    # Concatenate all subsets into a new DataFrame
    df = pd.concat(result_subsets, ignore_index=True)
    
    # Group so all sections payments for each schedule are together
    df = df.groupby(['Section Payment Schedule'])[['gross_amount', 'gocardless_fees', 'app_fees', 'net_amount']].sum().reset_index()
    
    # Sort so it is easier to read
    df = df.sort_values(by=['Section Payment Schedule'])

    # Display the result DataFrame
    return df

In [32]:
def manupulate_df(df):
    
    # Sort so it is easier to read
    df = df.sort_values(by=['Section Payment Schedule'])

    df['total_fees'] = df['gross_amount'] - df['net_amount']

    column_order = ['Section Payment Schedule', 'net_amount', 'total_fees', 'gross_amount', 'gocardless_fees',
       'app_fees']
    
    df = df[column_order]
    return df

In [33]:
def main():
    df = create_df()
    df = discover_amounts(df)
    df = manupulate_df(df)
    df.to_excel('outputs/output.xlsx', index=False)
    return df
df = main()
df

142 payments were found in this file


ModuleNotFoundError: No module named 'openpyxl'